<!-- WARNING: THIS FILE WAS AUTOGENERATED! DO NOT EDIT! -->

## ChatApp - Full Dependency Injection Architecture

This module provides a complete chat application with:
- FastHTML + WebSocket integration
- Spinner/loading states
- Full dependency injection
- Customizable components

In [0]:
#| echo: false
#| output: asis
show_doc(ChatAppConfig)

---

### ChatAppConfig

>      ChatAppConfig (app_title:str='Chat Application', page_title:str='Chat', i
>                     nitial_messages_factory:Optional[Callable[[],List[pylogue.
>                     session.Message]]]=None, bg_color:str='#1a1a1a',
>                     header_style:str='text-align: center; padding: 1em; color:
>                     white;', container_style:Optional[str]=None,
>                     ws_endpoint:str='/ws', markdown_enabled:bool=True,
>                     syntax_highlighting:bool=True,
>                     highlight_langs:List[str]=<factory>,
>                     spinner_css:Optional[str]=None)

*Configuration for ChatApp.*

In [0]:
#| echo: false
#| output: asis
show_doc(ChatApp)

---

### ChatApp

>      ChatApp (session_manager:pylogue.session.SessionManager,
>               chat_service:pylogue.service.ChatService,
>               renderer:pylogue.renderer.ChatRenderer,
>               config:Optional[__main__.ChatAppConfig]=None)

*Complete chat application with full dependency injection.

This is the main orchestration layer that ties together:
- Session management (state)
- Chat service (business logic)
- Renderer (presentation)
- FastHTML + WebSocket (infrastructure)*

## Factory Functions for Easy Setup

In [0]:
#| echo: false
#| output: asis
show_doc(create_default_chat_app)

---

### create_default_chat_app

>      create_default_chat_app (responder:pylogue.service.Responder,
>                               config:Optional[__main__.ChatAppConfig]=None,
>                               card:Optional[pylogue.cards.ChatCard]=None, sess
>                               ion_manager:Optional[pylogue.session.SessionMana
>                               ger]=None, error_handler:Optional[pylogue.servic
>                               e.ErrorHandler]=None,
>                               context_provider:Optional[Callable]=None)

*Factory function to create a ChatApp with sensible defaults.

Args:
    responder: The AI/LLM responder function
    config: Optional app configuration
    card: Optional custom ChatCard for styling
    session_manager: Optional custom session manager
    error_handler: Optional custom error handler
    context_provider: Optional context provider for responder

Returns:
    Configured ChatApp instance*

## Example: Creating a Chat App

In [0]:
#| echo: false
#| output: asis
show_doc(example_responder)

---

### example_responder

>      example_responder (message:str, context=None)

*Example echo responder with latency.*

In [7]:
# Example 1: Minimal setup with defaults
app = create_default_chat_app(responder=example_responder)

# To run: app.run()
print("✅ Created default chat app")

✅ Created default chat app


In [8]:
# Example 2: Custom configuration
custom_config = ChatAppConfig(
    app_title="My AI Assistant",
    page_title="AI Chat",
    bg_color="#0a0a0a",
    markdown_enabled=True,
    syntax_highlighting=True,
)

# Custom card styling
custom_card = ChatCard(
    user_color="#2d4a3e",
    assistant_color="#3d2a4a",
    user_emoji="👤",
    assistant_emoji="🤖",
    font_size="1.2em",
)

app = create_default_chat_app(
    responder=example_responder, config=custom_config, card=custom_card
)

print("✅ Created custom styled chat app")

✅ Created custom styled chat app


In [9]:
# Example 3: Full DI with custom components
from pylogue.service import ContextAwareResponder

# Custom responder that uses history
context_responder = ContextAwareResponder(max_history=10)


# Custom context provider
def history_context_provider(session: ChatSession):
    """Provide last N messages as context."""
    messages = session.get_messages()
    return messages[-5:] if len(messages) > 5 else messages


# Custom session manager (could be Redis, DB, etc.)
custom_session_manager = InMemorySessionManager()

# Assemble with full DI
chat_service = ChatService(
    responder=context_responder, context_provider=history_context_provider
)

renderer = ChatRenderer(card=ChatCard(user_color="#1a3a2a", assistant_color="#3a1a2a"))

config = ChatAppConfig(
    app_title="Context-Aware Assistant",
    initial_messages_factory=lambda: [
        Message(role="Assistant", content="Hello! I remember our conversation history.")
    ],
)

app = ChatApp(
    session_manager=custom_session_manager,
    chat_service=chat_service,
    renderer=renderer,
    config=config,
)

print("✅ Created fully customized DI chat app")

✅ Created fully customized DI chat app


## Testing Individual Components

In [10]:
# Test session management
manager = InMemorySessionManager()
session = manager.create_session("test-123")
session.add_message("User", "Hello")
print(f"Session messages: {session.get_message_dicts()}")

Session messages: [{'role': 'User', 'content': 'Hello', 'id': 'aa7212ac-7c89-4b42-9668-e462b6d73e89'}]


In [11]:
# Test chat service
from pylogue.service import ChatService

service = ChatService(responder=example_responder)
response = await service.process_message("Test message")
print(f"Service response: {response}")

Service response: Echo: Test message


In [12]:
# Test renderer with messages
from fasthtml.jupyter import render_ft

render_ft()

messages = [
    Message(role="User", content="Hello!"),
    Message(role="Assistant", content="Hi there! How can I help?"),
    Message(role="Assistant", content="", pending=True),
]

renderer = ChatRenderer()
renderer.render_messages(messages)

<div>
  <div id="chat-cards" class="chat-cards" style="display: flex; flex-direction: column; gap: 10px;">
    <div style="background: #272727; padding: 10px; font-size: 1.5em; width: 60%; align-self: center; text-align: right; border-radius: 1em; padding: 1.25em">
<span style="font-weight: bold; font-size: 1.1em; display: block; margin-bottom: 8px;"><u>🗣️ User: </u></span>      <div class="marked" style="white-space: pre-wrap;">Hello!</div>
    </div>
    <div style="background: #3B3B3B; padding: 10px; font-size: 1.5em; width: 60%; align-self: center; text-align: left; border-radius: 1em; padding: 1.25em">
<span style="font-weight: bold; font-size: 1.1em; display: block; margin-bottom: 8px;"><u>🕵️‍♂️ Assistant: </u></span>      <div class="marked" style="white-space: pre-wrap;">Hi there! How can I help?</div>
    </div>
    <div style="background: #3B3B3B; padding: 10px; font-size: 1.5em; width: 60%; align-self: center; text-align: left; border-radius: 1em; padding: 1.25em">
🕵️‍♂️ Assistant: <span class="spinner"></span>    </div>
  </div>
<script>if (window.htmx) htmx.process(document.body)</script></div>
